### Create an API session

In [63]:
import pandas as pd

from weaveapi.records import *
from weaveapi.options import *
from weaveapi.filter import *
from weaveapi.weaveh import *

WEAVE_CONFIG = "config/demo_ailineage_owner.config"
nodeApi, session = connect_weave_api(WEAVE_CONFIG)

data_collection = "vault"

token = "USDC"
decimals = 6

{"res":"ok","data":"pong 1690036891794"}


#### Check initial node owner balance

Bridge address: https://blockscout.chiadochain.net/address/0x54b82d06FaD03a788CC8840528038310DBD1b697

In [53]:
account = session.publicKey
reply = nodeApi.balance(session, account, "", token).get()
print("Custodial wallet", account, float(reply["data"]) / pow(10, decimals) if "data" in reply else 0)

Custodial wallet weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q 930010.0


### Create tables to store prompts

In [54]:
personas = [
    "solidity_engineer",
    "paris_guide",
    "pregnancy_nurse",
    "startup_advisor",
    "crypto_trader",
    "ethical_data_consultant"
]

#### Prompt proposals

In [55]:
for persona in personas:
    print(persona)
    layout = { 
        "columns": { 
            "id": { "type": "LONG", "isIndexed": True, "isUnique": True, "isNullable": False },
            "ts": { "type": "LONG" },
            "pubkey": { "type": "STRING" },
            "sig": { "type": "STRING" },
            "ip": { "type": "STRING" },
            "roles": { "type": "STRING" },
            "text": { "type": "STRING" }
        }, 
        "idColumnIndex": 0,  # Autogenerates IDs
        "timestampColumnIndex": 1, # Fills the column automatically with the network time
        "ownerColumnIndex": 2, # Fills the pubkey column automatically with the public key of the writer
        "signatureColumnIndex": 3, # Fills the column with an EdDSA signature of the record hash
        "sourceIpColumnIndex": 4, # Fills the column with an EdDSA signature of the record hash
        "allowedRolesColumnIndex": 5, # Allowed readers, specified by the writer
        "isLocal": False,
        "applyReadTransformations": True
    }

    table = persona + "_proposals"
    nodeApi.dropTable(session, data_collection, table).get()
    reply = nodeApi.createTable(session, data_collection, table, CreateOptions(False, False, layout)).get()
    print(reply)

solidity_engineer
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'solidity_engineer_proposals'}}
paris_guide
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'paris_guide_proposals'}}
pregnancy_nurse
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'pregnancy_nurse_proposals'}}
startup_advisor
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'startup_advisor_proposals'}}
crypto_trader
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'ac

#### Accepted prompts

In [56]:
for persona in personas:
    print(persona)
    layout = { 
        "columns": { 
            "id": { "type": "LONG", "isIndexed": True, "isUnique": True, "isNullable": False },
            "ts": { "type": "LONG" },
            "pubkey": { "type": "STRING" },
            "sig": { "type": "STRING" },
            "ip": { "type": "STRING" },
            "roles": { "type": "STRING" },
            "source": { "type": "STRING" },
            "text": { "type": "STRING" }
        }, 
        "idColumnIndex": 0,  # Autogenerates IDs
        "timestampColumnIndex": 1, # Fills the column automatically with the network time
        "ownerColumnIndex": 2, # Fills the pubkey column automatically with the public key of the writer
        "signatureColumnIndex": 3, # Fills the column with an EdDSA signature of the record hash
        "sourceIpColumnIndex": 4, # Fills the column with an EdDSA signature of the record hash
        "allowedRolesColumnIndex": 5, # Allowed readers, specified by the writer
        "isLocal": False,
        "applyReadTransformations": True
    }

    table = persona + "_prompts"
    nodeApi.dropTable(session, data_collection, table).get()
    reply = nodeApi.createTable(session, data_collection, table, CreateOptions(False, False, layout)).get()
    print(reply)

solidity_engineer
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'solidity_engineer_prompts'}}
paris_guide
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'paris_guide_prompts'}}
pregnancy_nurse
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'pregnancy_nurse_prompts'}}
startup_advisor
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'startup_advisor_prompts'}}
crypto_trader
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 

#### Superprompts

In [57]:
for persona in personas:
    print(persona)
    layout = { 
        "columns": { 
            "id": { "type": "LONG", "isIndexed": True, "isUnique": True, "isNullable": False },
            "ts": { "type": "LONG" },
            "pubkey": { "type": "STRING" },
            "sig": { "type": "STRING" },
            "ip": { "type": "STRING" },
            "roles": { "type": "STRING" },
            "text": { "type": "STRING" }
        }, 
        "idColumnIndex": 0,  # Autogenerates IDs
        "timestampColumnIndex": 1, # Fills the column automatically with the network time
        "ownerColumnIndex": 2, # Fills the pubkey column automatically with the public key of the writer
        "signatureColumnIndex": 3, # Fills the column with an EdDSA signature of the record hash
        "sourceIpColumnIndex": 4, # Fills the column with an EdDSA signature of the record hash
        "allowedRolesColumnIndex": 5, # Allowed readers, specified by the writer
        "isLocal": False,
        "applyReadTransformations": True
    }

    table = persona + "_superprompts"
    nodeApi.dropTable(session, data_collection, table).get()
    reply = nodeApi.createTable(session, data_collection, table, CreateOptions(False, False, layout)).get()
    print(reply)

solidity_engineer
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'solidity_engineer_superprompts'}}
paris_guide
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'paris_guide_superprompts'}}
pregnancy_nurse
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'pregnancy_nurse_superprompts'}}
startup_advisor
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'startup_advisor_superprompts'}}
crypto_trader
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'wea

#### Create table for compute lineage

In [64]:
layout = { 
    "columns": { 
        "id": { "type": "LONG", "isIndexed": True, "isUnique": True, "isNullable": False },
        "ts": { "type": "LONG" },
        "pubkey": { "type": "STRING" },
        "sig": { "type": "STRING" },
        "ip": { "type": "STRING" },
        "roles": { "type": "STRING" },
        "persona": { "type": "STRING" },
        "lineage": { "type": "STRING" }
    }, 
    "idColumnIndex": 0,  # Autogenerates IDs
    "timestampColumnIndex": 1, # Fills the column automatically with the network time
    "ownerColumnIndex": 2, # Fills the pubkey column automatically with the public key of the writer
    "signatureColumnIndex": 3, # Fills the column with an EdDSA signature of the record hash
    "sourceIpColumnIndex": 4, # Fills the column with an EdDSA signature of the record hash
    "allowedRolesColumnIndex": 5, # Allowed readers, specified by the writer
    "isLocal": False,
    "applyReadTransformations": True
}

table = "personas_lineage"
nodeApi.dropTable(session, data_collection, table).get()
reply = nodeApi.createTable(session, data_collection, table, CreateOptions(False, False, layout)).get()
print(reply)

{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'personas_lineage'}}


#### Write some sample prompt

In [60]:
toInsert = [
    [ "paris_guide_proposals", "You are a well trained Paris City guide that knows every attraction. You have to give a recommendation for a tourist such that it is both funny and interesting, based on the following input:"],
    [ "paris_guide_proposals", "As a Paris City Guide for foreigners you have to recommend the best place to visit in order to meet interesting people. What would that be, if the visitor wants"],
    [ "paris_guide_proposals", "You are an alternative city guide who knows the unbeaten tracks of Paris. You have to recommend things for people such that they will be pleasantly surprised" ]
    
]
table = 

text = 
records = Records(table, [
    [ None, None, None, None, None, None, text ]
])
reply = nodeApi.write(session, data_collection, records, WRITE_DEFAULT).get()
print(reply)


text = "As a Paris City Guide recommend me something to do"
records = Records(table, [
    [ None, None, None, None, None, None, text ]
])
reply = nodeApi.write(session, data_collection, records, WRITE_DEFAULT).get()
print(reply)

text = 
records = Records(table, [
    [ None, None, None, None, None, None, text ]
])
reply = nodeApi.write(session, data_collection, records, WRITE_DEFAULT).get()
print(reply)

{'res': 'ok', 'target': {'operationType': 'WRITE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'paris_guide_proposals'}, 'data': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q,rbmtntkzhI54Zu2icINzmN+vFIMhr6tK0ljUnq9hHaY=,3kYAnrUPsDkVbvjciNdD9kRpZMJ5rQbXUKcnHe4FTuJSijitT42gaZJfiJmWbhdXfBrgxH9USYujzCm7UM8V7cj6', 'ids': '15'}
{'res': 'ok', 'target': {'operationType': 'WRITE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'paris_guide_proposals'}, 'data': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q,lPbPQ/LFWA1A0SzlbF6tBP8Gg6uCveAwvMPEPYVM+Fg=,4PXHZv3heXo6V9VjYkZ4tVaAUTjkLPAafNJX1nPgJw37A1znPeCmfLudjVJPG4xUs8w6d1NF3b7aFsVy8TXZn4XF', 'ids': '16'}
{'res': 'ok', 'target': {'operationType': 'WRITE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'paris_guide_p

#### Test read proposed prompts

In [46]:
table = "paris_guide_proposals"

reply = nodeApi.read(session, data_collection, table, None, READ_DEFAULT_NO_CHAIN).get()
#print(reply)
df = pd.DataFrame(reply["data"])
display(df.tail())

sample_prompt = df.iloc[0]["text"]

,id,ts,pubkey,sig,ip,text,roles
0,1,1690018985189,weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioS...,"[{""sig"":{""recordsHash"":""rbmtntkzhI54Zu2icINzmN...",172.174.28.163,You are a well trained Paris City guide that k...,NaN
1,2,1690018985246,weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioS...,"[{""sig"":{""recordsHash"":""KLZkMkd8JyVGwOyhDCH7W6...",172.174.28.163,As a Paris City Guide recommend me something t...,NaN
2,3,1690018985289,weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioS...,"[{""sig"":{""recordsHash"":""CzriI5SNDKnG1qvUOak8U2...",172.174.28.163,As a Paris City Guide for foreigners you have ...,NaN
3,4,1690021401822,weaveqTYSfBMsu1zqzxizPy6gc9Wj3mNdvjkTqmYuRHFqVtcA,"[{""sig"":{""recordsHash"":""Pvm03Nb6o/YTY9QkKMYgId...",89.137.100.219,You are an alternative city guide who knows th...,*
4,5,1690021917107,weaveqTYSfBMsu1zqzxizPy6gc9Wj3mNdvjkTqmYuRHFqVtcA,"[{""sig"":{""recordsHash"":""GjvVh2Ta+jCJB7zHAtudxV...",89.137.100.219,test,*


#### Records have hashes and signatures from the source

In [9]:
display(json.loads(df.iloc[-1]["sig"]))

[{'sig': {'recordsHash': 'CzriI5SNDKnG1qvUOak8U2ClHg8a4skXp1EcP6hpbis=',
   'count': '3',
   'pubKey': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q',
   'prevRecordsHash': 'KLZkMkd8JyVGwOyhDCH7W6Mx71QhUj6pBuqjhJ4ZEsk=',
   'sig': '5EWTgwvyrag6v9bYhSvWNA6tLjXB1B1PbZnhn858gGEY1FukosYKMW8Q28GQWjcZnqxn1Ys5v4gLmUPWQtuZ6nYY'}}]

### Run compute task to accept prompts

In [51]:
timeoutSec = 300
params = {
    "persona": "paris_guide",
    "accept": [ 1, 3 ]
}
options = ComputeOptions(True, timeoutSec, 0, None, params)

### TODO. Leave auth for later
reply = nodeApi.compute(session, "gcr.io/weavechain/approve_prompts", options).get()
display(reply)

{'res': 'ok',
 'target': {'operationType': 'COMPUTE',
  'organization': 'weavedemo',
  'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q'},
 'data': {'inputHash': 'nMPqnwoRvHPp3mtXFP8JbRNs4bRjgiptVCovyyTTeZJ',
  'console': '{"res":"ok","data":"pong 1690029442440"}\n7\n{\'res\': \'ok\', \'target\': {\'operationType\': \'WRITE\', \'organization\': \'weavedemo\', \'account\': \'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q\', \'scope\': \'vault\', \'table\': \'paris_guide_prompts\'}, \'data\': \'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q,FkVUQ7UnnZ8x2Nz21qJs1tPGXnnQ7RmlQatLJSeXv3M=,4vmgzCKtL63v1B5RBQz92ZYT2gzS6WgtfZuEGbBJueigdj6dpL32q1Dsazqzujop9HqdADAAWit5VWtBwZrnAjnG\', \'ids\': \'9\'}\n{\'res\': \'ok\', \'target\': {\'operationType\': \'WRITE\', \'organization\': \'weavedemo\', \'account\': \'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q\', \'scope\': \'vault\', \'table\': \'paris_guide_prompts\'}, \'data\': \'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5io

#### Test read accepted prompts

In [32]:
table = "paris_guide_prompts"

reply = nodeApi.read(session, data_collection, table, None, READ_DEFAULT_NO_CHAIN).get()
#print(reply)
df = pd.DataFrame(reply["data"])
display(df.tail())

sample_prompt = df.iloc[0]["text"]
sample = df.iloc[0]

,id,ts,pubkey,sig,ip,roles,source,text
0,1,1690019451376,weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioS...,"[{""sig"":{""recordsHash"":""FkVUQ7UnnZ8x2Nz21qJs1t...",localhost,*,weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioS...,You are a well trained Paris City guide that k...
1,2,1690019451404,weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioS...,"[{""sig"":{""recordsHash"":""IbLKkVDQ96sV/EYgUXRvgA...",localhost,*,weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioS...,As a Paris City Guide for foreigners you have ...


### Run compute task to generate a superprompt

In [27]:
timeoutSec = 300
image = "gcr.io/weavechain/distil_prompt"

params = {
    "persona": "paris_guide",
    "scope": data_collection
}
options = ComputeOptions(True, timeoutSec, 0, None, params)

reply = nodeApi.compute(session, image, options).get()
display(reply["data"])

{'inputHash': '3NvhzYCRa8WPfCVvkGrX6XEc72qJrv8Uk6762VgNDxhY',
 'console': 'Connecting to node...\n{"res":"ok","data":"pong 1690019602117"}\nFetched params: {\'persona\': \'paris_guide\', \'scope\': \'vault\', \'WEAVE_CALLER_PUBKEY\': \'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q\'}\nFetched scope: vault\nFetched persona: paris_guide\nFetching prompts:  $$$ You are a well trained Paris City guide that knows every attraction. You have to give a recommendation for a tourist such that it is both funny and interesting, based on the following input:  $$$ As a Paris City Guide for foreigners you have to recommend the best place to visit in order to meet interesting people. What would that be, if the visitor wants \nFetched distilation_prompt: You will get instructions for summarizing many paragraphs into a single paragraph.\nYou will write an answer that\'s on average the same size as the different paragraphs taken one by one. Make sure that all important keywords are, in the end, in t

#### Read superprompt

In [33]:
table = "paris_guide_prompts"

res = nodeApi.read(session, data_collection, table, None, READ_DEFAULT_NO_CHAIN).get()
#print(reply)
df = pd.DataFrame(res["data"])

super_prompt = df.iloc[-1]["text"]

display(super_prompt)
display(df.tail(1))

'As a Paris City Guide for foreigners you have to recommend the best place to visit in order to meet interesting people. What would that be, if the visitor wants '

,id,ts,pubkey,sig,ip,roles,source,text
1,2,1690019451404,weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioS...,"[{""sig"":{""recordsHash"":""IbLKkVDQ96sV/EYgUXRvgA...",localhost,*,weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioS...,As a Paris City Guide for foreigners you have ...


#### Compute lineage

In [29]:
task_reply = reply["data"]

print("Compute Task Hash:", task_reply["computeHash"])
print("Input Hash:", task_reply["inputHash"])
print("Root Hashes:", task_reply.get("rootHashes"))
print("Output Hash:", task_reply["outputHash"])
print("Output Signature:", task_reply["outputSignatureTs"])

signature = task_reply["outputSignature"]
match = nodeApi.verifyLineageSignature(signature, task_reply.get("inputHash"), task_reply.get("computeHash"), task_reply.get("paramsHash"), task_reply["output"])
print("\nSignature Match:", match)

Compute Task Hash: 9njuvKmvCpypveTLfgKGtRHK2GZZHiBdYGmWuFgytSHW
Input Hash: 3NvhzYCRa8WPfCVvkGrX6XEc72qJrv8Uk6762VgNDxhY
Root Hashes: "{vault={paris_guide_prompts=DuisniJjpv8vpkRhAkiFHZw42KPhi36XRP83xUAoUzmL}}"
Output Hash: 2QQbaKNrH2KJbvyCCz2VuBi4ULusLseqGPMr6Av1AW7J
Output Signature: 26xwWm8xKvVXcKjKAbN48WmvJZVzejcz3HJbuXCh3U3KYqeFZkvfdFFWY5Wo5ZPE6k8JcJkMSU8LwdR2YDtSCnTk

Signature Match: True


### Check balance for writers (will need to use different accounts to have a payment happening)

In [61]:
account = sample["pubkey"]
reply = nodeApi.balance(session, account, "", token).get()
print("Custodial wallet:", session.publicKey, float(reply["data"]) / pow(10, decimals) if "data" in reply else 0)

Custodial wallet: weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q 0


### Blockchain anchors for immutability proofs (for Paris guide, accepted prompts)

#### Records have immutability proofs on blockchain

https://blockscout.chiadochain.net/address/0xb63c92907027FF85b428Af54B378d6D09688157B

In [35]:
reply = nodeApi.hashes(session, data_collection, table, None, READ_DEFAULT_NO_CHAIN).get()
#print(reply)
dh = pd.DataFrame([ [x, reply["data"][x]] for x in list(reply["data"]) ], columns=["ID", "Hash"])
display(dh)

,ID,Hash
0,1,LzzkXddC/6CQPrfZ2E6Uw5j443OVr1855BzsTGoIrSM=


#### Merkle Tree Proofs

#### Records have a different type of immutability proof offering the ability to check inclusion via merkle trees

https://blockscout.chiadochain.net/address/0x281912371C490C4818cC57afB8dF88114C6888C0

In [36]:
from binarytree import Node

def showtree(tree):
    prev = None
    root = None
    lvl = 1
    for l in tree.split(";"):
        level = l.split(",")
        print(lvl, level)
        lvl += 1
        nodes = []
        for i in range(len(level)):
            pidx = int(i / 2)
            node = Node(level[i][:3] + ".." + level[i][-3:])
            if root is None:
                root = node
            nodes.append(node)
            if prev is not None:
                parent = prev[pidx]
                if parent.left is None:
                    parent.left = node
                else:
                    parent.right = node
        prev = nodes
    
    print(root)

salt = "salt1234" # Same salt used for records hashes, this can be improved to have different salts for *each distinct writer*
digest = "SHA-256"

filter = Filter(None, None, None, None, [ "text" ])
reply = nodeApi.merkleTree(session, data_collection, table, filter, salt, digest, READ_DEFAULT_NO_CHAIN).get()
tree = reply["data"]["tree"]
rootHash = reply["data"]["rootHash"]
ts = reply["data"]["timestamp"]
rootHashSignature = reply["data"]["signature"]

print("Generated at", ts)
print("Root Hash", rootHash)
print("Signature", rootHashSignature)
print("")
showtree(tree)
# We've built a Merkle Tree at a specific time, signed by the node that created it.
# The Merkle Leaves are salted hashes of the data in the table. Binance did an enumeration of coins.

Generated at 1690019691477
Root Hash DuisniJjpv8vpkRhAkiFHZw42KPhi36XRP83xUAoUzmL
Signature 3cWdTnf915dM4g49TARQRvTcj9FoYeHoHUqDgv9Yw29bRmLu4wok7UNNaMtDKFR4XDvwkfLurn17rNi4yKarXoHB

1 ['DuisniJjpv8vpkRhAkiFHZw42KPhi36XRP83xUAoUzmL']
2 ['H6UdXXMF71En5drhGTEhUDkACdxSeEpLewXxbbUicRdz', 'DnVaHn9sMpgeDyrLdqnhVSjj7mNDksJNP6eMpNE7x2yC']

     ____Dui..zmL___
    /               \
H6U..Rdz          DnV..2yC



#### Check the root hash stored on chain

In [37]:
table = "paris_guide_prompts"

reply = nodeApi.rootHash(session, data_collection, table).get()
#print(reply)
data = reply["data"]
display(data)

chainRootHash = data["rootHash"]
print("\nMatching:", rootHash == chainRootHash) # data hashes are salted. The salt needs to match what's configured on the network to have a match

{'signature': '38z6wvRCEHjF9r5XX215hD16XuULC7MNmyWBZjCuUhKGpGVrrXbUtcwUBW7eAKnm7U4JJ7J9smNdrpvx1XDztXQS',
 'rootHash': 'DuisniJjpv8vpkRhAkiFHZw42KPhi36XRP83xUAoUzmL',
 'timestamp': '1690019694146'}


Matching: True


#### Verify root hash signature

In [38]:
signingKey = nodeApi.sigKey().get()["data"]
print("Node Public Key:", signingKey)

Node Public Key: GfHq2tTVk9z4eXgyXVRvnn4hnpsfgFCfen9n1KoqnLp9ANhnmj9BsdJvv9Gg


In [39]:
toSign = rootHash + " " + ts
#print(rootHash)
check = nodeApi.verifySignature(rootHashSignature, toSign)
print("Check signature from merkleTree() call:", check)

toSign = data["rootHash"] + " " + data["timestamp"]
check = nodeApi.verifySignature(data["signature"], toSign)
print("Check signature from blockchain:", check)

# Note, verifySignature isn't unique to Weavechain, it's EdDSA public key cryptography and the verification is done locally

Check signature from merkleTree() call: True
Check signature from blockchain: True


### Verify inclusion

#### With SHA-256 hash

Compute the hash of one of the articles

In [40]:
row = [ sample_prompt ]
recordHash = nodeApi.hashRecord(row, salt)
print(recordHash)

H6UdXXMF71En5drhGTEhUDkACdxSeEpLewXxbbUicRdz


Check the hash presence and validate the Merkle Tree

In [41]:
reply = nodeApi.verifyMerkleHash(session, tree, recordHash).get()
print(reply["data"])

true


Or get a Merkle Proof for the record hash and verify it

In [42]:
table = "paris_guide_prompts"

reply = nodeApi.merkleProof(session, data_collection, table, recordHash).get()
print(reply)
data = reply["data"]
display(data)

hashes = [ h.split(",") for h in data["proof"].split(";") ]
display(hashes)

proofSignature = data["signature"]
toSign = data["proof"] + " " + data["timestamp"] # the node signed the proof and the timestamp
check = nodeApi.verifySignature(proofSignature, toSign)
print(check)

{'res': 'ok', 'data': {'signature': '4zYeMLLy7FhTZgxJfkgUycEngEyiLuyN7GWyvLZUTmuthY724DgmCoDybZzAvLURsfLiLbhx3MTGhaa99hgMzq4B', 'rootHash': 'DuisniJjpv8vpkRhAkiFHZw42KPhi36XRP83xUAoUzmL', 'proof': 'H6UdXXMF71En5drhGTEhUDkACdxSeEpLewXxbbUicRdz,DnVaHn9sMpgeDyrLdqnhVSjj7mNDksJNP6eMpNE7x2yC', 'timestamp': '1690019703136'}}


{'signature': '4zYeMLLy7FhTZgxJfkgUycEngEyiLuyN7GWyvLZUTmuthY724DgmCoDybZzAvLURsfLiLbhx3MTGhaa99hgMzq4B',
 'rootHash': 'DuisniJjpv8vpkRhAkiFHZw42KPhi36XRP83xUAoUzmL',
 'proof': 'H6UdXXMF71En5drhGTEhUDkACdxSeEpLewXxbbUicRdz,DnVaHn9sMpgeDyrLdqnhVSjj7mNDksJNP6eMpNE7x2yC',
 'timestamp': '1690019703136'}

[['H6UdXXMF71En5drhGTEhUDkACdxSeEpLewXxbbUicRdz',
  'DnVaHn9sMpgeDyrLdqnhVSjj7mNDksJNP6eMpNE7x2yC']]

True


### Use persona

In [44]:
image = "gcr.io/weavechain/generate_content"
timeoutSec = 300
params = {
    "persona": "paris_guide",
    "prompt": "Please tell me where to spend my Sunday morning ",
    "scope": data_collection
}
options = ComputeOptions(True, timeoutSec, 0, None, params)

reply = nodeApi.compute(session, image, options).get()
display(reply["data"]["output"])
display(reply["data"])

"Certainly! If you're an art lover, I recommend starting your Sunday morning at the Louvre Museum. It opens at 9 AM and it's less crowded in the early hours. You can explore the museum at your own pace, marveling at works like the Mona Lisa and the Venus de Milo. \n\nIf you prefer a more relaxed morning, head to the Marais district. Start with a breakfast at a café, then take a leisurely stroll through the charming streets, popping into any boutiques that catch your eye. The Musée Picasso and the Place des Vosges are also in this area.\n\nFor a more nature-oriented morning, visit the Jardin du Luxembourg or the Bois de Boulogne. Both are beautiful parks where you can enjoy a picnic, go for a jog, or simply sit and watch the world go by. \n\nIf you're interested in history, you might want to visit the historic Montmartre district. You can visit the Sacré-Cœur Basilica, stroll through the artistic Place du Tertre, and visit the Musée de Montmartre.\n\nRemember, Paris is best enjoyed at a

{'inputHash': '99rEqBDABrTCjYshhYASV8URwyWPc6hNKdRFBAAH6Zy',
 'console': 'Connecting to node...\n{"res":"ok","data":"pong 1690019744778"}\nFetched params: {\'persona\': \'paris_guide\', \'scope\': \'vault\', \'prompt\': \'Please tell me where to spend my Sunday morning \', \'WEAVE_CALLER_PUBKEY\': \'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q\'}\nFetched scope: vault\nFetched persona: paris_guide\nFetching prompt: Please tell me where to spend my Sunday morning \nFetching superprompt: As a Paris City Guide, the role involves recommending the most captivating places for tourists to visit to meet intriguing individuals, ensuring the advice is both amusing and engaging. This entails having an in-depth knowledge of all the attractions in Paris and being able to provide guidance that aligns with a visitor\'s unique interests.\nFetching max_tokens: 3072\nFetched chat completion flag: True\nFetched model endine: gpt-4\nGot chatgpt response: Certainly! If you\'re an art lover, I recomme